In [327]:
# 사용할 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#한글 깨짐
plt.rc("font", family="Malgun Gothic")

In [328]:
# 파일 불러오기
test = pd.read_csv('data/test.csv', index_col='no')
train = pd.read_csv('data/train.csv', index_col='no')

## 예측할 값 : 개인 소득
- income : 50K 초과는 1, 50K 이하는 0 (소득)

### 컬럼 의미
- age - continuous.
- workclass(일 유형) : Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt(일련의 관측 결과를 바탕으로 인구조사국이 부여하는 개인의 가중치): continuous.
- education(교육수준) : Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num(교육수준 번호) : continuous.
- marital-status(결혼 상태) : Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation(직업) : Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship(가족관계) : Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race(인종) : White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex(성별) : Female, Male.
- capital-gain(자본 이익) : continuous.
- capital-loss(자본 손실) : continuous.
- hours-per-week(주당 근무 시간) : continuous.
- native-country(국적) : United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

### value값 띄어쓰기 제거
- value_counts()로 찍어보니 뭔가 이상하다
- 앞에 띄어쓰기가 있네? 띄어쓰기 없애준다

In [329]:
# lambda 함수
# lambda 매개변수 : 표현식
for i in train.columns : 
    if train[f'{i}'].dtype == 'object' :
        train[f'{i}'] = train[f'{i}'].apply(lambda x: x.replace(' ', ''))
    else :
        train[f'{i}'] = train[f'{i}']
        
for i in test.columns : 
    if test[f'{i}'].dtype == 'object' :
        test[f'{i}'] = test[f'{i}'].apply(lambda x: x.replace(' ', ''))
    else :
        test[f'{i}'] = test[f'{i}']

### ? 채우기
- ?는 최빈값으로 채우는게 제일 낫다고 판단

In [330]:
train['occupation'].value_counts()

Prof-specialty       3724
Craft-repair         3632
Exec-managerial      3609
Adm-clerical         3440
Sales                3292
Other-service        2975
Machine-op-inspct    1811
?                    1668
Transport-moving     1393
Handlers-cleaners    1249
Farming-fishing       888
Tech-support          865
Protective-serv       603
Priv-house-serv       146
Armed-Forces           10
Name: occupation, dtype: int64

In [331]:
work_occ = train[(train['workclass'] == '?') & (train['occupation'] == '?')].shape[0]
work = train[train['workclass'] == '?'].shape[0]
print("workclass에서 ?인 애들은 occupation에서도 ? 냐?",work_occ == work)

# 일 안하니까 쪽팔려서 안적은거 아니야?
# workclass는 Never-worked로 바꿔버림..

workclass에서 ?인 애들은 occupation에서도 ? 냐? True


In [332]:
train[(train['workclass'] != 'Never-worked')]['occupation'].value_counts()
#일안하는 애들은 occupation에 적을게 없네? 그럼 ? 냅둬보자

Prof-specialty       3724
Craft-repair         3632
Exec-managerial      3609
Adm-clerical         3440
Sales                3292
Other-service        2975
Machine-op-inspct    1811
?                    1663
Transport-moving     1393
Handlers-cleaners    1249
Farming-fishing       888
Tech-support          865
Protective-serv       603
Priv-house-serv       146
Armed-Forces           10
Name: occupation, dtype: int64

In [333]:
title=['Private', 'Self-emp-not-inc', 'Local-gov', '?',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
cover_title=['Private', 'Self-emp-not-inc', 'Local-gov', 'Never-worked',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
title_dict = dict(zip(title, cover_title))
title_dict
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] =test['workclass'].map(title_dict)

"title=['Private', 'Self-emp-not-inc', 'Local-gov', '?',\n       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']\ncover_title=['Private', 'Self-emp-not-inc', 'Local-gov', 'Never-worked',\n       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']\ntitle_dict = dict(zip(title, cover_title))\ntitle_dict\ntrain['workclass'] = train['workclass'].map(title_dict)\ntest['workclass'] =test['workclass'].map(title_dict)"

In [334]:
# native-country의 ?의 갯수는 400대로 적고 United-States의 값이 압도적으로 많기 때문에 최빈값으로 채움
title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       '?', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
cover_title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'United-States', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
title_dict = dict(zip(title, cover_title))
title_dict
train['native-country'] = train['native-country'].map(title_dict)

test['native-country'] = test['native-country'].map(title_dict)

In [335]:
title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',
       'Armed-Forces']
cover_title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',
       'Armed-Forces']
title_dict = dict(zip(title, cover_title))
title_dict
train['occupation'] = train['occupation'].map(title_dict)
test['occupation'] = test['occupation'].map(title_dict)

"title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',\n       'Tech-support', 'Transport-moving', 'Farming-fishing',\n       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',\n       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',\n       'Armed-Forces']\ncover_title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',\n       'Tech-support', 'Transport-moving', 'Farming-fishing',\n       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',\n       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',\n       'Armed-Forces']\ntitle_dict = dict(zip(title, cover_title))\ntitle_dict\ntrain['occupation'] = train['occupation'].map(title_dict)\ntest['occupation'] = test['occupation'].map(title_dict)"

In [336]:
# education과 education-num은 동일한 내용
# education-num은 숫자라 나중에 카테고리화 하기 쉬우니 남겨두고 education을 지우기
train = train.drop('education',axis=1)
test = test.drop('education',axis=1)

In [337]:
# marital_status에서는 결혼한 상태인지 아닌지만 보면 될 것으로 판단
marital_status = ['Divorced', 'Never-married', 'Married-civ-spouse', 'Separated',
       'Married-spouse-absent', 'Widowed', 'Married-AF-spouse']
conver_marital_status = ['not-married', 'not-married', 'married', 'not-married', 'not-married', 'not-married','married']
marital_status_dict=dict(zip(marital_status, conver_marital_status))

train['marital-status']=train['marital-status'].map(marital_status_dict)
test['marital-status']=test['marital-status'].map(marital_status_dict)

In [338]:
# husband와 wife빼고는 큰 의미 없는 듯해서 그냥 나두고 둘은 married로 치환
train['relationship'].unique()
title=['Not-in-family', 'Husband', 'Unmarried', 'Other-relative',
       'Own-child', 'Wife']
cover_title=['Not-in-family', 'Married', 'Unmarried', 'Other-relative',
       'Own-child', 'Married']
title_dict = dict(zip(title, cover_title))
train['relationship'] = train['relationship'].map(title_dict)
test['relationship'] = test['relationship'].map(title_dict)

In [339]:
# 교육수준은 크게 저,중,고로 나눌 수 있는데, 중간이 많기 때문에 그냥 4분할 함
bins = [0,4,8,12,16] # 범위지정
labels = ['e1', 'e2', 'e3', 'e4'] # 라벨지정
train['education-num'] = pd.cut(train['education-num'], bins=bins, labels = labels)
test['education-num'] = pd.cut(test['education-num'], bins=bins, labels=labels)

In [340]:
# 나이는 처음엔 3분할 했으나 너무 크게 묶이는 거 같아 20대 별로 재분할
bins = [i for i in range(0, 101,20)]
labels = ['age' + str(i) for i in range(0, 100,20)] # 라벨지정
train['age'] = pd.cut(train['age'], bins=bins, labels = labels)
test['age'] = pd.cut(test['age'], bins=bins, labels=labels)

In [341]:
# workclass는 사기업, 사업가, 공무원, 거지로 나눔
title=['Private', 'Self-emp-not-inc', 'Local-gov', 'State-gov',
       'Self-emp-inc', 'Federal-gov', 'Without-pay', 'Never-worked']
cover_title=['Private', 'Self-emp', 'gov', 'gov',
       'Self-emp', 'gov', 'beggar', 'beggar']
title_dict = dict(zip(title, cover_title))
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] = test['workclass'].map(title_dict)

In [342]:
# United-States가 압도적으로 많기 때문에 따로 나두고, 나머지는 대륙별로 묶기
native_country = ['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Columbia', 'Hungary', 'Portugal', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
conver_native_country = ['United-States', 'North America', 'Asia', 'Europe', 'North America',
                         'North America', 'North America', 'North America', 'Asia', 'Europe',
                         'North America', 'Asia', 'Asia', 'North America', 'Europe',
                         'Asia', 'South America', 'North America', 'Asia', 'Europe',
                        'North America', 'Europe', 'Asia', 'Asia', 'North America',
                         'South America', 'Europe', 'Europe', 'South America', 'Asia',
                         'Europe', 'Asia', 'North America', 'Asia', 'Europe',
                         'Asia', 'South America', 'Europe', 'North America', 'Europe', 'Europe']

native_country_dict = dict(zip(native_country, conver_native_country))
native_country_dict
train['native-country']=train['native-country'].map(native_country_dict)
test['native-country']=test['native-country'].map(native_country_dict)

In [343]:
# fnlwgt는 없는게 더 잘나오길래 지워버림
train = train.drop('fnlwgt',axis=1)
test= test.drop('fnlwgt',axis=1)

In [344]:
# 문제/답 분리
X_train = train.loc[:,:'native-country']
y_train = train['income']

"# 문제/답 분리\nX_train = train.loc[:,:'native-country']\ny_train = train['income']"

In [345]:
X_train = pd.get_dummies(X_train) # 0과 1로만 이루어진 열을 생성
X_test = pd.get_dummies(test)

'X_train = pd.get_dummies(X_train) # 0과 1로만 이루어진 열을 생성\nX_test = pd.get_dummies(test)'

In [362]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

### 모델 학습시키고 결과 제출
- knn, decisionTree등은 성능 안 높아지길래 폐기하고
- 안배운 모델은 없나? 하고 찾아보던 중 lgbm과 catboost모델 등을 찾아서 사용하기로 함

https://hyunicecream.tistory.com/5

#### 그리드 서치

In [23]:
"""ada_model = AdaBoostClassifier()
ada_param = {'n_estimators' : [50,150,250],
             'learning_rate' : [0.5,1]
            }
ada_grid = GridSearchCV(ada_model,ada_param,cv = 3)
ada_grid.fit(X_train,y_train)
print('최고 평균 정확도 수치: {:.4f}'.format(ada_grid.best_score_))
print('최적 하이퍼 파라미터: ', ada_grid.best_params_)
# 그리드서치 바탕으로 하이퍼파라미터 조정
final_ada_model = ada_grid.best_estimator_"""
final_ada_model = AdaBoostClassifier(learning_rate= 1,
                                    n_estimators = 250)
final_ada_model.fit(X_train,y_train)
pre = final_ada_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_final_ada_model.csv', index = False)

- 최고 평균 정확도 수치: 0.8628
- 최적 하이퍼 파라미터:  {'learning_rate': 1, 'n_estimators': 250}

- 최고 평균 정확도 수치: 0.8657
- 최적 하이퍼 파라미터:  {'learning_rate': 0.01, 'max_depth': 50, 'n_estimators': 800, 'num_leaves': 100}

In [24]:
"""lgbm_model = LGBMClassifier()
lgbm_param = {"learning_rate" : [0.01,0.1,0.2,0.3,0.4,0.5],
              "max_depth" : [25, 50, 75],
              "num_leaves" : [100,300,500,900,1200],
              "n_estimators" : [100, 200, 300,500,800,1000]
              }
lgbm_grid = GridSearchCV (lgbm_model, lgbm_param, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
lgbm_grid.fit(X_train,y_train)
print('최고 평균 정확도 수치: {:.4f}'.format(lgbm_grid.best_score_))
print('최적 하이퍼 파라미터: ', lgbm_grid.best_params_)

final_lgbm_model = lgbm_grid.best_estimator_"""
final_lgbm_model = LGBMClassifier(learning_rate=0.01,
                                 max_depth = 50,
                                 n_estimators=800,
                                 num_leaves=100)
final_lgbm_model.fit(X_train, y_train)
pre = final_lgbm_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_final_lgbm_model.csv', index = False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001364 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 349
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473


In [25]:
"""cat_model = CatBoostClassifier()
cat_param = {"depth" : [4,6,8,10],
          "iterations" : [250,100,500,1000],
          "learning_rate" : [0.001,0.01,0.1,0.2,0.3], 
          "l2_leaf_reg" : [2,5,10,20,30],
          "border_count" : [254]
          }
cat_grid = GridSearchCV (cat_model, cat_param, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
cat_grid.fit(X_train,y_train)
print('최고 평균 정확도 수치: {:.4f}'.format(cat_grid.best_score_))
print('최적 하이퍼 파라미터: ', cat_grid.best_params_)"""

final_cat_model = CatBoostClassifier(verbose = 200)
final_cat_model.fit(X_train, y_train)
pre = final_cat_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_final_cat_model.csv', index = False)

Learning rate set to 0.043583
0:	learn: 0.6493672	total: 147ms	remaining: 2m 26s
200:	learn: 0.2878280	total: 1.67s	remaining: 6.65s
400:	learn: 0.2728914	total: 3.2s	remaining: 4.78s
600:	learn: 0.2648222	total: 4.64s	remaining: 3.08s
800:	learn: 0.2587812	total: 6.21s	remaining: 1.54s
999:	learn: 0.2539103	total: 7.76s	remaining: 0us


In [26]:
xgb_model = XGBClassifier()
xgb_param = {"max_depth": [10,30,50],
             "min_child_weight" : [1,3,6,10],
             "n_estimators": [200,300,500,1000]
            }
xgb_grid = GridSearchCV (xgb_model, xgb_param, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
xgb_grid.fit(X_train,y_train)
print('최고 평균 정확도 수치: {:.4f}'.format(xgb_grid.best_score_))
print('최적 하이퍼 파라미터: ', xgb_grid.best_params_)

final_xgb_model = xgb_grid.best_estimator_
final_xgb_model.fit(X_train, y_train)
pre = final_xgb_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_final_xgb_model.csv', index = False)

In [27]:
gbm_model = GradientBoostingClassifier()
gbm_param = {"max_depth" : [4,6,8,10],
             "learning_rate" : [0.01,0.1,0.3,0.5],
             "n_estimators" : [100,300,500]
            }
gbm_grid = GridSearchCV (gbm_model, gbm_param, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gbm_grid.fit(X_train,y_train)
print('최고 평균 정확도 수치: {:.4f}'.format(gbm_grid.best_score_))
print('최적 하이퍼 파라미터: ', gbm_grid.best_params_)

final_gbm_model = gbm_grid.best_estimator_
final_gbm_model.fit(X_train, y_train)
pre = final_gbm_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_final_gbm_model2.csv', index = False)

In [28]:
lreg_model = LogisticRegression()
lreg_param = {'C' : [1.0, 3, 5, 7, 10],
             'max_iter': [50, 100, 300, 500,700, 800]
            }
lreg_grid = GridSearchCV (lreg_model, lreg_param, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
lreg_grid.fit(X_train,y_train)
print('최고 평균 정확도 수치: {:.4f}'.format(lreg_grid.best_score_))
print('최적 하이퍼 파라미터: ', lreg_grid.best_params_)
final_lreg_model = lreg_grid.best_estimator_
final_lreg_model.fit(X_train, y_train)
pre = final_lreg_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_final_lreg_model.csv', index = False)

In [29]:
final_voting_model = VotingClassifier(
                        estimators=[
                        ('final_ada_mode',final_ada_model),
                        ('final_lgbm_model',final_lgbm_model),
                        ('final_cat_model',final_cat_model),
                        ('final_gbm_model',final_gbm_model),
                        ('final_xgb_model',final_xgb_model),
                        ('final_lreg_model',final_lreg_model)
                        ],
                        voting='soft'
                        )

In [30]:
# 학습
final_voting_model.fit(X_train,y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 349
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
Learning rate set to 0.043583
0:	learn: 0.6493672	total: 9.76ms	remaining: 9.75s
200:	learn: 0.2878280	total: 1.44s	remaining: 5.74s
400:	learn: 0.2728914	total: 2.88s	remaining: 4.29s
600:	learn: 0.2648222	total: 4.36s	remaining: 2.89s
800:	learn: 0.2587812	total: 5.83s	remaining: 1.45s
999:	learn: 0.2539103	total: 7.46s	remaining: 0us


VotingClassifier(estimators=[('final_ada_mode',
                              AdaBoostClassifier(learning_rate=1,
                                                 n_estimators=250)),
                             ('final_lgbm_model',
                              LGBMClassifier(learning_rate=0.01, max_depth=50,
                                             n_estimators=800,
                                             num_leaves=100)),
                             ('final_cat_model',
                              <catboost.core.CatBoostClassifier object at 0x0000016567607B50>),
                             ('final_gbm_model',
                              GradientBoostingClassifier(max_depth=6)),
                             ('final...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=30,
                                            max_leaves=None, min_child_weight=6,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('final_lreg_model',
                              LogisticRegression(C=10, max_iter=500))],
                 voting='soft')

In [31]:
# 예측
pre = final_voting_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_final_voting_model.csv', index = False)

voting 모델끼리 voting 시키기

In [32]:
final_voting_model1 = VotingClassifier(
                        estimators=[
                        ('final_ada_mode',final_ada_model),
                        ('final_lgbm_model',final_lgbm_model),
                        ('final_cat_model',final_cat_model)
                        ],
                        voting='soft'
                        )
# 학습
final_voting_model1.fit(X_train,y_train)
# 예측
pre = final_voting_model1.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_real_final_voting_model1.csv', index = False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 349
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
Learning rate set to 0.043583
0:	learn: 0.6493672	total: 7.31ms	remaining: 7.3s
200:	learn: 0.2878280	total: 1.71s	remaining: 6.79s
400:	learn: 0.2728914	total: 3.32s	remaining: 4.96s
600:	learn: 0.2648222	total: 4.84s	remaining: 3.21s
800:	learn: 0.2587812	total: 6.42s	remaining: 1.59s
999:	learn: 0.2539103	total: 8.05s	remaining: 0us


In [34]:
final_voting_model3 = VotingClassifier(
                        estimators=[
                        ('final_gbm_model',final_gbm_model),
                        ('final_xgb_model',final_xgb_model),
                        ('final_lreg_model',final_lreg_model)
                        ],
                        voting='soft'
                        )
# 학습
final_voting_model3.fit(X_train,y_train)
pre = final_voting_model3.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_real_final_voting_model3.csv', index = False)

In [35]:
real_final_voting_model = VotingClassifier(
                                            estimators=[
                                            ('final_voting_model1',final_voting_model1),                                            
                                            ('final_voting_model3',final_voting_model3)
                                            ],
                                            voting='soft'
                                            )
# 학습
real_final_voting_model.fit(X_train,y_train)
pre = real_final_voting_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('real_real_real_real_real_real_real_real_final_voting_model.csv', index = False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000798 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 349
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
Learning rate set to 0.043583
0:	learn: 0.6493672	total: 7.76ms	remaining: 7.76s
200:	learn: 0.2878280	total: 1.33s	remaining: 5.29s
400:	learn: 0.2728914	total: 2.65s	remaining: 3.96s
600:	learn: 0.2648222	total: 3.96s	remaining: 2.63s
800:	learn: 0.2587812	total: 5.28s	remaining: 1.31s
999:	learn: 0.2539103	total: 6.68s	remaining: 0us
[LightGBM] [Warning] Foun